In [1]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

### Models and Configs

In [2]:
import yaml

# Load YAML config
with open("config/model_config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Unpack general
general_cfg = config["general"]
num_periods = general_cfg["num_periods"]
num_scenarios = general_cfg["num_scenarios"]
num_generators = general_cfg["num_generators"]
num_tiers = general_cfg["num_tiers"]
num_storage = general_cfg["num_storage"]

# Unpack paths
paths = config["data_paths"]
gen_csv_path = paths["generator_csv"]
storage_csv_path = paths["storage_csv"]
demand_csv_path = paths["demand_csv"]

# Unpack solver config
solver_cfg = config["solver"]
solver_name = solver_cfg["name"]
solver_exec = solver_cfg.get("executable")
solver_options = solver_cfg.get("options", {})

import sys
sys.path.append('./src')

from DAFOModel import DAFOModel
from RTSimModel import RTSimModel

Process Data

In [3]:
class SystemDataProcessor:
    def __init__(self, gen_csv_path, storage_csv_path, demand_csv_path):
        self.gen_csv_path = gen_csv_path
        self.storage_csv_path = storage_csv_path
        self.demand_csv_path = demand_csv_path
        self.gen_data = None
        self.storage_data = None
        self.demand_data = None

    def load_data(self):
        try:
            self.gen_data = pd.read_csv(self.gen_csv_path).head(5)
            self.storage_data = pd.read_csv(self.storage_csv_path).head(1)
            print(f"Generator and Storage data loaded successfully. {len(self.gen_data)} generators and {len(self.storage_data)} storages found.")
            self.demand_data = pd.read_csv(self.demand_csv_path)
            print(f"Demand data loaded successfully. {len(self.demand_data)} periods found.")
            return True
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            return False

    def process_gen_data(self):
        id_col = 'GEN UID'
        column_mapping = {
            # 'Ramp Rate MW/Min': '', 
            # 'Fuel Price $/MMBTU': '', 
            # 'VOM': '',
            'PMax MW': 'CAP',
            'Ramp Rate MW/Min': 'RR',
        }

        relevant_cols = list(column_mapping.keys())
        existing_cols = [col for col in relevant_cols if col in self.gen_data.columns]
        selected_cols = [id_col] + existing_cols
        gen_data_filtered = self.gen_data[selected_cols].copy()

        rename_dict = {col: column_mapping[col] for col in column_mapping if col in gen_data_filtered.columns}
        gen_data_filtered.rename(columns=rename_dict, inplace=True)

        original_ids = gen_data_filtered[id_col].tolist()
        gen_id_mapping = {original_id: i+1 for i, original_id in enumerate(original_ids)}
        gen_data_filtered[id_col] = gen_data_filtered[id_col].map(gen_id_mapping)
        
        gen_data_filtered.set_index(id_col, inplace=True)

        # if 'Fuel Price $/MMBTU' in gen_data_filtered.columns and 'HR_avg_0' in self.gen_data.columns:
        #     gen_data_filtered['VC'] = self.gen_data['Fuel Price $/MMBTU'] * self.gen_data['HR_avg_0'] / 1000.0
        #     if 'VOM' in gen_data_filtered.columns:
        #         gen_data_filtered['VC'] += gen_data_filtered['VOM']
        
        # TODO - CALCULATE VC, VCUP, VCDN
        gen_data_filtered['VC'] = 20
        gen_data_filtered['VCUP'] = 20
        gen_data_filtered['VCDN'] = 20

        params = ['CAP', 'RR','VC','VCUP','VCDN']
        gen_data_dict = {}

        for param in params:
            if param in gen_data_filtered.columns:
                param_dict = {}
                for gen_idx in gen_data_filtered.index:
                    param_dict[gen_idx] = gen_data_filtered.loc[gen_idx, param]
                gen_data_dict[param] = param_dict
            
        return gen_data_dict

    def process_storage_data(self):
        if self.storage_data is None or self.storage_data.empty:
            return {}
            
        try:
            id_col = 'GEN UID'

            column_mapping = {
                'Max Volume GWh': 'E_MAX',
                'Rating MVA': 'P_MAX',
                'Initial Volume GWh': 'E0',
                # 'Storage Roundtrip Efficiency': 'ETA'
            }

            relevant_cols = list(column_mapping.keys())
            existing_cols = [col for col in relevant_cols if col in self.storage_data.columns]
                
            selected_cols = [id_col] + existing_cols
            storage_data_filtered = self.storage_data[selected_cols].copy()

            rename_dict = {col: column_mapping[col] for col in column_mapping if col in storage_data_filtered.columns}
            storage_data_filtered.rename(columns=rename_dict, inplace=True)

            original_ids = storage_data_filtered[id_col].tolist()
            storage_id_mapping = {original_id: i+1 for i, original_id in enumerate(original_ids)}
            storage_data_filtered[id_col] = storage_data_filtered[id_col].map(storage_id_mapping)
        
            storage_data_filtered.set_index(id_col, inplace=True)

            storage_data_dict = {}

            if 'E_MAX' in storage_data_filtered.columns:
                storage_data_dict['E_MAX'] = {
                    ## REMINDER: REMOVE THIS, just for testing
                    # storage_idx: float(storage_data_filtered.at[storage_idx, 'E_MAX'])
                    storage_idx: 0
                    for storage_idx in storage_data_filtered.index
                }
                
            if 'P_MAX' in storage_data_filtered.columns:
                storage_data_dict['P_MAX'] = {
                    storage_idx: float(storage_data_filtered.loc[storage_idx, 'P_MAX'])
                    for storage_idx in storage_data_filtered.index
                }
            
            # check charging and discharging efficiency
            storage_data_dict['ETA_CH'] = {
                storage_idx: 0.9    
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['ETA_DCH'] = {
                storage_idx: 0.9
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['STORAGE_COST'] = {
                storage_idx: 0.9
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['E0'] = {
                storage_idx: 0.0
                for storage_idx in storage_data_filtered.index
            }
            storage_data_dict['E_FINAL'] = {
                storage_idx: 0.0
                for storage_idx in storage_data_filtered.index
            }

            return storage_data_dict
            
        except Exception as e:
            print(f"Error processing storage data: {str(e)}")
            return {}
    
    def process_demand_data(self, num_periods=24):
        try:
            demand_data = self.demand_data.head(num_periods)
            demand_dict = demand_data.set_index(demand_data.index + 1)['1'].to_dict()
            return demand_dict
        except Exception as e:
            print(f"Error processing demand data: {str(e)}")
            return {}     

    def prepare_pyomo_data(self, num_periods, num_scenarios, num_tiers):
        """Prepare the data for the Pyomo model."""
        if self.gen_data is None:
            success = self.load_data()
            if not success:
                return None
                
        gen_data_dict = self.process_gen_data()
        storage_data_dict = self.process_storage_data()
        demand_data_dict = self.process_demand_data(num_periods)
        
        num_generators = len(gen_data_dict.get('CAP', {}))
        num_storage = len(storage_data_dict.get('E_MAX', {}))
        
        sets = {
            'T': {None: list(range(1, num_periods + 1))},
            'S': {None: list(range(1, num_scenarios + 1))},
            'G': {None: list(range(1, num_generators + 1))},
            'R': {None: list(range(1, num_tiers + 1))},
            'B': {None: list(range(1, num_storage + 1))}
        }
        
        reda_data = {} # dont need REDA for DA
        RE_data = np.random.uniform(0, 100, size=(num_scenarios, num_periods))
        RE_dict = {(s, t): RE_data[s-1][t-1] for s in range(1, num_scenarios+1) for t in range(1, num_periods+1)}
        
        fo_params = {
            # Manual values
            'D1': {None: 5},
            'D2': {None: 550.0},
            'PEN': {None: 2000},
            'PENDN': {None: 0},
            'smallM': {None: 0.01},
            # check probabilities
            'probTU': {1: 0.2, 2: 0.4, 3: 0.6, 4: 0.8},
            'probTD': {1: 0.8, 2: 0.6, 3: 0.4, 4: 0.2}
        }
        
        pyomo_data = {}
        pyomo_data.update(sets)
        # pyomo_data.update(gen_data_dict)
        pyomo_data.update(storage_data_dict)
        pyomo_data.update(fo_params)
        pyomo_data['DEMAND'] = demand_data_dict
        
        # TODO - CALCULATE REDA, RE
        pyomo_data['CAP'] = {1: 50, 2: 10, 3: 10, 4: 10, 5: 10}
        pyomo_data['RR'] = {1: 50, 2: 10, 3: 10, 4: 10, 5: 10}
        pyomo_data['VC'] = {1: 20, 2: 35, 3: 50, 4: 60, 5: 70}
        pyomo_data['VCUP'] = {1: 20, 2: 35, 3: 50, 4: 60, 5: 70}
        pyomo_data['VCDN'] = {1: 20, 2: 35, 3: 50, 4: 60, 5: 70}
        pyomo_data['RE'] = {
            (1, 1): 131, (1, 2): 131,
            (2, 1): 141, (2, 2): 141,
            (3, 1): 155, (3, 2): 155,
            (4, 1): 165, (4, 2): 165,
            (5, 1): 172, (5, 2): 172
        }
        # pyomo_data['RE'] = {
        #     (1, 1): 131, (1, 2): 131,
        # }
        pyomo_data['DEMAND'] = {1: 200, 2:200}
        pyomo_data['REDA'] = reda_data
        # pyomo_data['RE'] = RE_dict

        pyomo_data = {None: pyomo_data} # reformat for pyomo

        # TODO: UPDATE REQUIRED PARAMETERS
        required_params = [
            'CAP', 'VC', 'VCUP', 'VCDN', 'RR',
            'E_MAX', 'P_MAX', 'ETA_CH', 'ETA_DCH', 'E0', 'E_FINAL', 'STORAGE_COST',
            'D1', 'D2', 'PEN', 'PENDN', 'smallM', 'probTU', 'probTD',
            'DEMAND', 'REDA', 'RE'
        ]
        
        missing_params = [param for param in required_params if param not in pyomo_data[None]]
        if missing_params:
            print(f"Warning: Missing parameters: {missing_params}")
        
        print(storage_data_dict)
        return pyomo_data

In [4]:
system_data = SystemDataProcessor(gen_csv_path, storage_csv_path, demand_csv_path)
pyomo_system_data = system_data.prepare_pyomo_data(
    num_periods=num_periods,
    num_scenarios=num_scenarios,
    num_tiers=num_tiers
)

dafo_model = DAFOModel(
    num_periods=num_periods,
    num_scenarios=num_scenarios,
    num_generators=num_generators,
    num_tiers=num_tiers,
    num_storage=num_storage
)
da_instance = dafo_model.create_instance(pyomo_system_data)

opt = pyo.SolverFactory(solver_name, executable=solver_exec)
result = opt.solve(da_instance, tee=solver_options.get("tee", False))

Generator and Storage data loaded successfully. 5 generators and 1 storages found.
Demand data loaded successfully. 8784 periods found.
{'E_MAX': {1: 0}, 'P_MAX': {1: 200.0}, 'ETA_CH': {1: 0.9}, 'ETA_DCH': {1: 0.9}, 'STORAGE_COST': {1: 0.9}, 'E0': {1: 0.0}, 'E_FINAL': {1: 0.0}}

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.2.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2024.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\hanbo\AppData\Local\Temp\tmpnnsz1ud4.cplex.log' open.
CPLEX> Problem 'C:\Users\hanbo\AppData\Local\Temp\tmpgsgaguou.pyomo.lp' read.
Read time = 0.00 sec. (0.02 ticks)
CPLEX> Problem name         : C:\Users\hanbo\AppData\Local\Temp\tmpgsgaguou.pyomo.lp
Objective sense      : Minimize
Variables            :     1

In [5]:
def _extract_da_outputs(i, pyomo_system_data):
    # Create DataFrame for energy values
    Energy = pd.DataFrame()
    # Fix xDA access to handle time dimension
    E_grid_data = {(g): sum(i.xDA[g,t].value for t in i.T) for g in i.G}
    
    # Store the energy values
    Energy['en'] = [i.xDA[g,t].value for g in i.G for t in i.T]
    
    # Directly store values in dictionary
    xDA_data = {(g,t): i.xDA[g,t].value for g in i.G for t in i.T}
    
    # Store reserve requirements
    temp = {t: i.rgDA[t].value for t in i.T}
    
    # Calculate total cost and price
    Total = pd.DataFrame()
    # Calculate total cost across all time periods
    Total.at['cost','DA'] = sum(sum(i.VC[g] * i.xDA[g,t].value for g in i.G) for t in i.T)
    # Calculate total price from dual values
    Total.at['price','DA'] = sum(i.dual[i.Con3[t]] for t in i.T)
    
    # Handle demand data
    E_grid_data = {(R): v.value for (R), v in i.hdu.items()}
    demand = pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hdu"])
    E_grid_data = {(R): v.value for (R), v in i.hdd.items()}
    demand['hdd'] = pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hdd"])
        
    # Handle two-dimensional variables hsu and hsd
    hsu_data = {}
    hsd_data = {}
    for r in i.R:
        for g in i.G:
            for t in i.T:
                hsu_data[(r,g,t)] = i.hsu[r,g,t].value
                hsd_data[(r,g,t)] = i.hsd[r,g,t].value

    df = pd.DataFrame.from_dict(hsu_data, orient="index", columns=["hsu"])
    df['hsd'] = pd.DataFrame.from_dict(hsd_data, orient="index", columns=["hsd"])
    df['R'] = [k[0] for k in df.index]
    df['G'] = [k[1] for k in df.index]
    df['T'] = [k[2] for k in df.index]
    
    # Initialize empty lists to store prices
    up_prices = []
    down_prices = []

    # Collect prices for each time period and reserve tier
    for t in i.T:
        for r in i.R:
            up_prices.append(i.dual[i.Con4UP[r,t]])
            down_prices.append(i.dual[i.Con4DN[r,t]])

    # Create DataFrame with the collected prices
    Prices = pd.DataFrame({
        'up': up_prices,
        'down': down_prices,
        'T': [t for t in i.T for r in i.R],  # repeat each time period for each reserve tier
        'R': [r for t in i.T for r in i.R]   # repeat reserve tiers for each time period
    })
    
    # Calculate Gross Margins
    Gross_margins = pd.DataFrame()
    
    # Calculate energy margins using reshaping to avoid broadcasting errors
    # First create an array of VC values
    vc_array = np.array([i.VC[g] for g in i.G])
    
    # Calculate price component
    price_component = sum(i.dual[i.Con3[t]] for t in i.T)
    
    # Create energy arrays that are properly shaped
    energy_values = []
    for g in i.G:
        total_energy = sum(i.xDA[g,t].value for t in i.T)
        energy_values.append(total_energy)
    
    # Now calculate the margins
    Gross_margins["en"] = (price_component - vc_array) * np.array(energy_values)
    
    # Calculate reserve margins for each tier
    for tier in range(1,5):
        # Filter for specific tier and time period
        tier_data = df[df["R"]==tier]
        
        # If you have multiple time periods, you might want to aggregate or select a specific one
        # For example, to get first time period:
        tier_data = tier_data[tier_data["T"]==1].reset_index(drop=True)
        
        # Now calculate the margins
        vcup_array = np.array([i.VCUP[k] for k in i.G])
        price_component = Prices.at[tier-1,"up"] - np.multiply(vcup_array, i.probTU[tier])
        temp2 = np.multiply(tier_data["hsu"], price_component)
        
        temp2.index = range(1,6)
        Gross_margins["up"+str(tier)] = temp2
    
    # Extract storage decisions from DA stage
    p_ch_DA_data = {(b,t): i.p_ch[b,t].value for b in i.B for t in i.T}
    p_dch_DA_data = {(b,t): i.p_dch[b,t].value for b in i.B for t in i.T}

    dataRT = {None:{
        'RE': pyomo_system_data[None]["RE"],
        'CAP': pyomo_system_data[None]["CAP"],
        'VC': pyomo_system_data[None]["VC"],
        'VCUP': pyomo_system_data[None]["VCUP"],
        'VCDN': pyomo_system_data[None]["VCDN"],
        'DEMAND': pyomo_system_data[None]["DEMAND"],
        'D1': pyomo_system_data[None]["D1"],
        'D2': pyomo_system_data[None]["D2"],
        # 'prob':{1: 0.2},
        # probability of each scenario
        'prob':{1: 0.2, 2:0.2, 3:0.2, 4:0.2, 5:0.2},
        'RR': pyomo_system_data[None]["RR"],
        'xDA':xDA_data,
        'PEN':pyomo_system_data[None]["PEN"],
        'PENDN':pyomo_system_data[None]["PENDN"],
        'REDA': temp,  # Changed from nested structure to direct assignment
        'DAdr': {t: i.d[t].value for t in i.T},

        # storage parameters
        'E_MAX': pyomo_system_data[None]["E_MAX"],
        'P_MAX': pyomo_system_data[None]["P_MAX"],
        'ETA_CH': pyomo_system_data[None]["ETA_CH"],
        'ETA_DCH': pyomo_system_data[None]["ETA_DCH"],
        'STORAGE_COST': pyomo_system_data[None]["STORAGE_COST"],
        'E0': pyomo_system_data[None]["E0"],
        'E_FINAL': pyomo_system_data[None]["E_FINAL"],

        # storage decisions
        'p_ch_DA': p_ch_DA_data,
        'p_dch_DA': p_dch_DA_data
    }}
    
    # Return additional data along with dataRT
    return dataRT, Total, df, demand, Energy, Prices, Gross_margins

In [6]:
dataRT, Total, df, demand, Energy, Prices, Gross_margins = _extract_da_outputs(da_instance, pyomo_system_data)

dafo_model = RTSimModel(
    num_periods=num_periods,
    num_scenarios=num_scenarios, 
    num_generators=num_generators,
    num_storage=num_storage
)

iRT = dafo_model.create_instance(dataRT)
opt.solve(iRT)

RTmargins = pd.DataFrame()
for s in iRT.S:
    margins_for_scenario = []
    for g in iRT.G:
        gen_margin = 0
        for t in iRT.T:
            # Correct indexing for Con3 with both s and t
            dual_value = iRT.dual[iRT.Con3[s, t]]
            vc_component = iRT.prob[s] * dataRT[None]["VC"][g]
            gen_adjustment = iRT.xup[s, g, t].value - iRT.xdn[s, g, t].value
            
            margin = (dual_value - vc_component) * gen_adjustment
            gen_margin += margin
            
        margins_for_scenario.append(gen_margin)
    
    RTmargins[s] = margins_for_scenario
    
RTmargins.index = range(1, len(RTmargins) + 1)

RTpayoffs = pd.DataFrame()
for s in iRT.S: 
    if s < 5:
        for t in iRT.T:
            payoffs = []
            for g in iRT.G:
                mask = (df["R"] >= s) & (df["G"] == g)
                hsu_sum_for_g = df.loc[mask, "hsu"].sum()
                price_component = iRT.dual[iRT.Con3[s, t]] * hsu_sum_for_g
                cost_component = iRT.prob[s] * dataRT[None]["VCUP"][g] * hsu_sum_for_g
                payoff = -(price_component - cost_component)
                payoffs.append(payoff)
            
            RTpayoffs[f"UP{s}_t{t}"] = payoffs
            
    if s > 1:
        for t in iRT.T:
            payoffs = []
            for g in iRT.G:
                mask = (df["R"] < s) & (df["G"] == g)
                hsd_sum_for_g = df.loc[mask, "hsd"].sum()
                price_component = iRT.dual[iRT.Con3[s, t]] * hsd_sum_for_g
                cost_component = iRT.prob[s] * dataRT[None]["VCDN"][g] * hsd_sum_for_g
                payoff = price_component - cost_component
                payoffs.append(payoff)
            
            RTpayoffs[f"DN{s}_t{t}"] = payoffs

RTpayoffs.index = range(1, len(RTpayoffs) + 1)

Total = pd.DataFrame(index=['cost', 'price', 'unmet_demand'], 
                    columns=['DA'] + [f't{t}' for t in iRT.T])

for s in iRT.S:
    for t in iRT.T:
        # Compute costs for each time period
        up_costs = sum(dataRT[None]["VCUP"][g] * iRT.xup[s, g, t].value for g in iRT.G)
        down_costs = sum(dataRT[None]["VCDN"][g] * (-iRT.xdn[s, g, t].value) for g in iRT.G)
        
        Total.at['cost', f't{t}'] = iRT.prob[s] * (up_costs + down_costs)
        Total.at['price', f't{t}'] = iRT.dual[iRT.Con3[s, t]]
        
        # Unmet demand for each time period
        d_value = iRT.d[s, t].value
        D1_value = dataRT[None]["D1"][None]
        D2_value = dataRT[None]["D2"][None]
        Total.at['unmet_demand', f't{t}'] = iRT.prob[s] * (D1_value * d_value + D2_value * d_value * d_value)

# Calculate price convergence for each time period
price_convergence = pd.Series(index=iRT.T)
for t in iRT.T:
    price_convergence[t] = Total.at['price', 'DA'] - Total[f't{t}'].loc['price']

# maintain time dimension
premium_convergence = pd.DataFrame()
for t in iRT.T:
    premiums_t = (Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('up')]].sum().sum() +
                 Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('down')]].sum().sum())
    
    premium_convergence[f'UP_t{t}'] = (Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('up')]].sum(axis=1) +
                                      RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.endswith(f'_t{t}')]].sum(axis=1))
    
    premium_convergence[f'DN_t{t}'] = (Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('down')]].sum(axis=1) +
                                      RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.endswith(f'_t{t}')]].sum(axis=1))

premium_convergence.index.name = 'Generator'
premium_convergence.index = premium_convergence.index + 1

Total_margin = pd.DataFrame()
for k in range(1, 1):
    for t in iRT.T:
        # Basic generator margins for each time period
        Total_margin[f'gen_t{t}'] = (iRT.prob[k] * Gross_margins.sum(axis=1) + 
                                    RTmargins[k] + 
                                    RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.endswith(f'_t{t}')]].sum(axis=1))
        
        # Renewable energy margins for each time period
        re_adjustments = iRT.rgup[k, t].value - iRT.rgdn[k, t].value
        dual_value = iRT.dual[iRT.Con3[k, t]]
        da_dual_value = Total.at['price', 'DA']
        
        Total_margin.at['RE', f't{t}'] = (dual_value * re_adjustments +
                                         iRT.prob[k] * dataRT[None]["REDA"][t] * da_dual_value - 
                                         iRT.prob[k] * premiums)
        
        # Demand response margins for each time period
        d_value_rt = iRT.d[k, t].value
        d_value_da = dataRT[None]["DAdr"][t]
        D1_value = dataRT[None]["D1"][None]
        D2_value = dataRT[None]["D2"][None]
        
        Total_margin.at['DR', f't{t}'] = (dual_value * d_value_rt + 
                                         iRT.prob[k] * d_value_da * da_dual_value -
                                         iRT.prob[k] * (D1_value * (d_value_rt + d_value_da)) -
                                         iRT.prob[k] * (D2_value * (d_value_rt + d_value_da) * (d_value_rt + d_value_da)))

# Display results
print("\nRT Margins:")
print(round(RTmargins, 2))

print("\nRT Payoffs:")
print(round(RTpayoffs, 2))

print("\nSystem Metrics:")
print(round(Total, 2))

print("\nPremium Convergence:")
print(round(premium_convergence, 2))

print("\nTotal Margins:")
print(round(Total_margin, 2))

print("\nFO Supply AWARDS:")
print(round(df, 2))

print("\nFO Demand AWARDS:")
print(round(demand, 2))

print("\nDA Energy:")
print(round(Energy, 2))

print("\nPrices:")
print(round(Prices, 2))

ERROR: Constructing component 'E_MAX' from data={1: 0} failed:
        RuntimeError: Failed to set value for param=E_MAX, index=1, value=0.
    	source error message="Index '1' is not valid for indexed component
    	'E_MAX'"


RuntimeError: Failed to set value for param=E_MAX, index=1, value=0.
	source error message="Index '1' is not valid for indexed component 'E_MAX'"